In [ ]:
from specsanalyzer.core import SpecsAnalyzer
from specsanalyzer.convert import convert_image
import matplotlib.pyplot as plt

In [ ]:
spa = SpecsAnalyzer()

In [ ]:
print(spa)

In [ ]:
import numpy as np
ar = np.ones((10,10), float)
convert_image(ar, 15, 10, 1, 2, {})

In [ ]:
! pwd

In [ ]:
import pandas as pd
tsv_data = pd.read_csv('../tests/data/Scan1232.tsv', sep='\t')
plt.imshow(tsv_data)

In [ ]:
spa = SpecsAnalyzer(config="../tests/data/config/config.yaml")

In [ ]:
print(spa)

In [ ]:
import pandas as pd
tsv_data = pd.read_csv('../tests/data/Scan1232_filtered2.tsv', sep='\t')
plt.imshow(tsv_data)

In [ ]:
import pandas as pd
tsv_data = pd.read_csv('../tests/data/Scan1232_converted.tsv', sep='\t')
plt.imshow(tsv_data)